In [ ]:
import cv2
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from numpy import dot
from numpy import linalg as la
from numpy.linalg import norm
import matplotlib
from matplotlib import pyplot as plt
from collections import Counter
import time
import random

from sklearn.decomposition import TruncatedSVD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Crearea matricei de antrenare
A = np.zeros([10304,320])
index = 0
pathSursa="/content/drive/MyDrive/Proiecte"

for i in range(1,41):
  pathPersoane = pathSursa +"/s" + str(i) + "/"
  for j in range(1,9):
    pathPozaAntrenare = pathPersoane + str(j) + '.pgm'
    # citim poza ca matrice 112 x 92:
    pozaAntrenare = np.array(cv2.imread(pathPozaAntrenare,0))
    # vectorizam poza:
    pozaVect = pozaAntrenare.reshape(10304,)
    A[:,index] = pozaVect
    index=index+1

In [ ]:
#Algoritmul NN

def AlgoritmulNN(A, pozaCautata, norma):
  distante = np.zeros(len(A[0]))

  if(norma == "L1"):
    for i in range(0,len(distante)):
      distante[i] = np.linalg.norm((A[:,i]-pozaCautata), ord=1)

  elif(norma == "L2"):
    for i in range(0,len(distante)):
      distante[i] = la.norm(A[:,i]-pozaCautata)

  elif(norma == "LINF"):
    for i in range(0,len(distante)):
      distante[i] = np.linalg.norm((A[:,i]-pozaCautata), ord=np.inf)

  elif(norma == "LCOS"):
    for i in range(0,len(distante)):
      distante[i] = 1 - np.inner(A[:,i], pozaCautata)/(norm(A[:,i])*norm(pozaCautata))

  else:
    print("Nume norma incorect. Incercati L1, L2, LINF sau LCOS")
    return -1
    

  pozitia = np.argmin(distante) # returneaza indicele la care se afla cea mai mica distanta
  
  return pozitia # pozitia la care se afla imaginea gasita

# **Algoritmul Eigenfaces**

In [ ]:
def Eigenfaces(A, pozaCautata, k):
  #Preprocesare
  media = np.mean(A, axis = 1) 
  #B = A # backup

  A = (A.T - media).T

  L = np.dot(A.T, A)

  d, v = np.linalg.eig(L)

  v = np.dot(A, v)


  indici = np.argsort(d) 

  top_k_indici = indici[::-1][:k]

  HQPB = v[:,top_k_indici]


  proiectii = np.dot(A.T, HQPB)

In [ ]:
def EigenfacesTSVD(A, pozaCautata, k):
  #Preprocesare
  media = np.mean(A, axis = 1) 

  A = (A.T - media).T

  
  svd = TruncatedSVD(k) #k E{20,...,100}
  A_transf = svd.fit_transform(A.T)
  s = svd.singular_values_

  U = A_transf/s
  E = U*A_transf

  A_transf = U*E
  

  S = np.diag(s)
  VT = svd.components_

In [ ]:
#Afisarea rezultatului 
pozaCautata = (np.array(cv2.imread('/content/drive/MyDrive/Proiecte/s5/9.pgm',0))).reshape(10304,)
pozaCopie = pozaCautata

E = [20,40,60,80,100]

for k in E:
  print("Pentru k =", k)

  timp_start = time.perf_counter()

  rezultat_EIG = Eigenfaces(A, pozaCautata , k)

  timp_fin = time.perf_counter()
  timp_cautare = timp_fin - timp_start
  print("Folosind eig:", timp_cautare)



  timp_start = time.perf_counter()

  rezultat_SVD = EigenfacesTSVD(A, pozaCautata , k)

  timp_fin = time.perf_counter()
  timp_cautare = timp_fin - timp_start
  print("Folosind svd:", timp_cautare)

  print()


Pentru k = 20
Folosind eig: 0.28273667200028285
Folosind svd: 0.4835763169994607

Pentru k = 40
Folosind eig: 0.2746028869996735
Folosind svd: 0.6113768819996039

Pentru k = 60
Folosind eig: 0.27519946899974457
Folosind svd: 0.8051794220000374

Pentru k = 80
Folosind eig: 0.2985631599995031
Folosind svd: 1.0183248590001313

Pentru k = 100
Folosind eig: 0.2723861479998959
Folosind svd: 1.138669416999619

